# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Display sample data
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,La Rioja,ES,42.2500,-2.5000,40.78,86,55,3.51,broken clouds
1,Hilo,US,19.7297,-155.0900,77.25,94,0,5.75,light rain
2,Bambous Virieux,MU,-20.3428,57.7575,77.25,88,75,18.41,moderate rain
3,Hobart,AU,-42.8794,147.3294,48.06,85,100,18.41,light intensity shower rain
4,Dali,CN,25.7000,100.1833,44.17,74,41,4.23,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# 5. Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you want for your trip? "))

# 5. Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you want for your trip? "))

What is the minimum temperature you want for your trip? 70
What is the maximum temperature you want for your trip? 75


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
                        
# Display sample data
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,Lodja,CD,-3.4833,23.4333,70.09,95,100,1.45,overcast clouds
6,Mocuba,MZ,-16.8392,36.9856,74.95,94,100,0.78,light rain
37,Una,IN,20.8167,71.0333,71.71,54,2,7.25,clear sky
38,Biloela,AU,-24.4167,150.5000,74.16,60,36,3.27,scattered clouds
41,Huntsville,US,34.7304,-86.5861,70.68,95,100,8.05,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

clean_travel_cities.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,Lodja,CD,-3.4833,23.4333,70.09,95,100,1.45,overcast clouds
6,Mocuba,MZ,-16.8392,36.9856,74.95,94,100,0.78,light rain
37,Una,IN,20.8167,71.0333,71.71,54,2,7.25,clear sky
38,Biloela,AU,-24.4167,150.5000,74.16,60,36,3.27,scattered clouds
41,Huntsville,US,34.7304,-86.5861,70.68,95,100,8.05,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country","Max Temp",  "Current Description","Lat", "Lng"]].copy()

# Display sample data
hotel_df.head()


# # Use the city_data list to create a new pandas DataFrame.
# city_data_df = pd.DataFrame(city_data)
# city_data_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
5,Lodja,CD,70.09,overcast clouds,-3.4833,23.4333
6,Mocuba,MZ,74.95,light rain,-16.8392,36.9856
37,Una,IN,71.71,clear sky,20.8167,71.0333
38,Biloela,AU,74.16,scattered clouds,-24.4167,150.5000
41,Huntsville,US,70.68,overcast clouds,34.7304,-86.5861


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Lodja,CD,70.09,overcast clouds,-3.4833,23.4333,
6,Mocuba,MZ,74.95,light rain,-16.8392,36.9856,
37,Una,IN,71.71,clear sky,20.8167,71.0333,
38,Biloela,AU,74.16,scattered clouds,-24.4167,150.5000,
41,Huntsville,US,70.68,overcast clouds,34.7304,-86.5861,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Lodja,CD,70.09,overcast clouds,-3.4833,23.4333,
6,Mocuba,MZ,74.95,light rain,-16.8392,36.9856,Phama Lodge
37,Una,IN,71.71,clear sky,20.8167,71.0333,Hotel Harbhole
38,Biloela,AU,74.16,scattered clouds,-24.4167,150.5000,Silo Motor Inn Biloela
41,Huntsville,US,70.68,overcast clouds,34.7304,-86.5861,Embassy Suites by Hilton Huntsville


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
# Display sample data
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Mocuba,MZ,74.95,light rain,-16.8392,36.9856,Phama Lodge
37,Una,IN,71.71,clear sky,20.8167,71.0333,Hotel Harbhole
38,Biloela,AU,74.16,scattered clouds,-24.4167,150.5000,Silo Motor Inn Biloela
41,Huntsville,US,70.68,overcast clouds,34.7304,-86.5861,Embassy Suites by Hilton Huntsville
44,East London,ZA,70.74,clear sky,-33.0153,27.9116,Tu Casa


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label= False)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [14]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [15]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Phama Lodge</dd>\n<dt>City</dt><dd>Mocuba</dd>\n<dt>Country</dt><dd>MZ</dd>\n<dt>Current Weather</dt><dd>light rain and 74.95 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Harbhole</dd>\n<dt>City</dt><dd>Una</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Current Weather</dt><dd>clear sky and 71.71 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Silo Motor Inn Biloela</dd>\n<dt>City</dt><dd>Biloela</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 74.16 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Embassy Suites by Hilton Huntsville</dd>\n<dt>City</dt><dd>Huntsville</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 70.68 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Tu Casa</dd>\n<dt>City</dt><dd>East London</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>clear sky and 70.74 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hospitality Carnarvon</dd>\

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [16]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
6,-16.8392,36.9856
37,20.8167,71.0333
38,-24.4167,150.5000
41,34.7304,-86.5861
44,-33.0153,27.9116
46,-24.8667,113.6333
60,13.7574,7.9874
71,19.3003,97.9685
74,-8.7537,148.2534
83,10.9333,108.1000


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [17]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Create a figure to add the Google map as a layer
fig.add_layer(marker_layer)

# Display the figure containing the map
fig


Figure(layout=FigureLayout(height='420px'))